<a href="https://colab.research.google.com/github/SeongGwan-a/Project-CAU_Data_Analysis/blob/main/%EA%B3%A8%EB%AA%A9%EB%8C%80%EC%9E%A5%ED%8C%80_%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%BD%94%EB%93%9C2(%ED%81%AC%EB%A1%A4%EB%A7%81).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install selenium

In [ ]:
import os
import pandas as pd
import numpy as np
import random
import requests
import time
import re
from bs4 import BeautifulSoup 
from pandas import DataFrame
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

In [ ]:
# 사전에 운영체제에 맞는 크롬드라이버 설치 필요 
# 자신의 드라이버 경로 지정하기
# https://chromedriver.chromium.org/downloads
driver = webdriver.Chrome('chromedriver')
time.sleep(1)
input("자동으로 실행되는 크롬을 전체화면으로 설정해주세요.\n설정후 엔터를 입력하세요.\n")
# 자동으로 실행되는 크롬 창을 전체화면으로 설정해주세요.

In [ ]:
### 골목 식당 상권별 출현회차
# -> 종료 회차 기준
# 용산구 해방촌 20회
# 성동구 성수동 뚝섬 25회
# 강동구 성내동 38회
# 서대문구 포방터시장 44회
# 용산 청파동 하숙골목 49회
# 회기동 벽화골목 54회
# 둔촌동 88회
# 정릉 아리랑시장 92회
# 홍제동 문화촌 골목 104회
# 공릉동 기찻길 골목 109회
# 도봉구 창동 골목 133회
# 중곡동 시장 앞 골목 138회

times_viedo=[20,25,38,44,49,54,88,92,104,109,133,138]

In [ ]:
#회차 검색
for t in range(len(times_viedo)):
    times=times_viedo[t]
    url = 'https://tv.naver.com/search?query="백종원의 골목식당" '+str(times)+"회&page=1" 
    driver.get(url)
    time.sleep(1)
    
    # 영상 개수
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    a=[k.get_text() for k in soup.select('div.tit_box')]
    num_clip = list(map(int, re.findall("\d+",a[0])))[0]
    
    # 회차별 리뷰 크롤링
    review = []
    
    for nc in range(1, num_clip+1):
        driver.find_element_by_xpath('//*[@id="clip_list"]/div[{}]/div/a'.format(nc)).click()
        time.sleep(2)
        
        driver.find_element_by_xpath('//*[@id="playerArea"]').click()

        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        a=[k.get_text() for k in soup.select('div.u_cbox_paginate')]
        b=[k.get_text() for k in soup.select('div.u_cbox_head')]
        # 댓글 없는 겨우
        if np.max(list(map(int, re.findall("\d+",b[0])))) == 0 :
            driver.get(url)
            time.sleep(1)
            continue
        # 댓글이 하나인 경우
        elif np.max(list(map(int, re.findall("\d+",a[0])))) == 1 :
            max=1
        
        else : 
          # 댓글 페이지의 최대 max
          driver.find_element_by_xpath('//*[@class="u_cbox_next u_cbox_next_end"]').click()
          time.sleep(1)
          html = driver.page_source
          soup = BeautifulSoup(html, 'lxml')
          a=[k.get_text() for k in soup.select('div.u_cbox_page_wrap')]
          max = np.max(list(map(int, re.findall("\d",a[0]))))
          driver.find_element_by_xpath('//*[@class="u_cbox_pre u_cbox_pre_end"]').click()
          time.sleep(1)

        #max 만큼 실행
        for i in range(0,max):
            if i%5==0 and i//5>0:
                driver.find_element_by_xpath('//*[@class="u_cbox_next"]').click()
                time.sleep(0.5)
            else :
                driver.find_element_by_xpath('//*[@class="u_cbox_page"][{}]'.format(i%5 + 1)).click()
                time.sleep(0.5)

            html= driver.page_source
            soup= BeautifulSoup(html, 'lxml')
            a=[k.get_text() for k in soup.select('div.u_cbox_text_wrap')]
            for i in range(len(a)):
                review.append(a[i])

        driver.get(url)
        time.sleep(1)
        
    # 회차별 댓글 모음
    pd.DataFrame(np.array(review).flatten()).to_csv("review_{}.csv".format(times_viedo[t]), index=False, header = False)
    print(str(times_viedo[t])+"번째 회차 댓글 모음 csv 파일 저장 완료")

20번째 회차 댓글 모음 csv 파일 저장 완료
25번째 회차 댓글 모음 csv 파일 저장 완료
38번째 회차 댓글 모음 csv 파일 저장 완료
44번째 회차 댓글 모음 csv 파일 저장 완료
49번째 회차 댓글 모음 csv 파일 저장 완료
54번째 회차 댓글 모음 csv 파일 저장 완료
88번째 회차 댓글 모음 csv 파일 저장 완료
92번째 회차 댓글 모음 csv 파일 저장 완료
104번째 회차 댓글 모음 csv 파일 저장 완료
109번째 회차 댓글 모음 csv 파일 저장 완료
133번째 회차 댓글 모음 csv 파일 저장 완료
138번째 회차 댓글 모음 csv 파일 저장 완료


In [ ]:
# 상도동 회차 댓글 크롤링 (143번째 회차)
times=143
url = 'https://tv.naver.com/search?query="백종원의 골목식당" '+str(times)+"회&page=1" 
driver.get(url)
time.sleep(1)

# 영상 개수
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')
a=[k.get_text() for k in soup.select('div.tit_box')]
num_clip = list(map(int, re.findall("\d+",a[0])))[0]

# 회차별 리뷰 크롤링
review = []

for nc in range(1, num_clip+1):
    driver.find_element_by_xpath('//*[@id="clip_list"]/div[{}]/div/a'.format(nc)).click()
    time.sleep(2)

    driver.find_element_by_xpath('//*[@id="playerArea"]').click()

    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    a=[k.get_text() for k in soup.select('div.u_cbox_paginate')]
    b=[k.get_text() for k in soup.select('div.u_cbox_head')]
    # 댓글 없는 겨우
    if np.max(list(map(int, re.findall("\d+",b[0])))) == 0 :
        driver.get(url)
        time.sleep(1)
        continue
    # 댓글이 하나인 경우
    elif np.max(list(map(int, re.findall("\d+",a[0])))) == 1 :
        max=1

    else : 
      # 댓글 페이지의 최대 max
      driver.find_element_by_xpath('//*[@class="u_cbox_next u_cbox_next_end"]').click()
      time.sleep(1)
      html = driver.page_source
      soup = BeautifulSoup(html, 'lxml')
      a=[k.get_text() for k in soup.select('div.u_cbox_page_wrap')]
      max = np.max(list(map(int, re.findall("\d",a[0]))))
      driver.find_element_by_xpath('//*[@class="u_cbox_pre u_cbox_pre_end"]').click()
      time.sleep(1)

    #max 만큼 실행
    for i in range(0,max):
        if i%5==0 and i//5>0:
            driver.find_element_by_xpath('//*[@class="u_cbox_next"]').click()
            time.sleep(0.5)
        else :
            driver.find_element_by_xpath('//*[@class="u_cbox_page"][{}]'.format(i%5 + 1)).click()
            time.sleep(0.5)

        html= driver.page_source
        soup= BeautifulSoup(html, 'lxml')
        a=[k.get_text() for k in soup.select('div.u_cbox_text_wrap')]
        for i in range(len(a)):
            review.append(a[i])

    driver.get(url)
    time.sleep(1)

# 회차별 댓글 모음
pd.DataFrame(np.array(review).flatten()).to_csv("review_{}.csv".format(times), index=False, header = False)
print(str(times)+"번째 회차 댓글 모음 csv 파일 저장 완료")

143번째 회차 댓글 모음 csv 파일 저장 완료
